## 학습된 모델 불러오기

In [ ]:
from peft import PeftModel, PeftConfig, AutoPeftModelForCausalLM
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import AutoPeftModelForCausalLM

device = "cuda:0"
base_model = 'davidkim205/komt-mistral-7b-v1'

# Load base model(Mistral 7B)
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)

config = PeftConfig.from_pretrained("beta_1")
model = AutoModelForCausalLM.from_pretrained(base_model, quantization_config=bnb_config, device_map={"": 0})
model = PeftModel.from_pretrained(model, "beta_1")



# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True) #model_max_length=2048


tokenizer.padding_side = 'left'
tokenizer.pad_token = tokenizer.unk_token
tokenizer.add_eos_token = False
model.config.use_cache = True
model.eval()

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/1.67k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Lin

##  Train, Test data pre-processing

In [ ]:
from tqdm import tqdm
import re
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd

def process_korean_string(input_string):
    pattern = re.compile(r'\b(?:또한|그리고)\b')
    split_string = pattern.split(input_string)
    processed_segments = [segment.replace(',', '').strip() for segment in split_string]
    return processed_segments

def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b) if norm_a != 0 and norm_b != 0 else 0

def preprocess_train(train):
    sent_model = SentenceTransformer("jhgan/ko-sbert-sts")

    train['encoded_Q1'] = train['질문_1'].apply(lambda x : sent_model.encode(x))
    train['encoded_Q2'] = train['질문_2'].apply(lambda x : sent_model.encode(x))
    train['concatenated_답변'] = train.apply(lambda row: '\n'.join(row[['답변_1', '답변_2', '답변_3', '답변_4', '답변_5']]), axis=1)
    return train

def test_split_question_and_RAG(train, test):
    sent_model = SentenceTransformer("jhgan/ko-sbert-sts")

    test['질문_split'] = test['질문'].apply(process_korean_string)
    encoded = []

    for q_list in tqdm(test['질문_split']):
        tmp_list = []
        for q in q_list:
            if len(q) > 0:
                tmp_list.append(sent_model.encode(q))

        encoded.append(tmp_list)
    test['encoded'] = encoded

    idx_list = []
    for q_list in tqdm(encoded):
        tmp_list = []
        for q in q_list:
            max_score = 0
            max_index = 0
            for idx in range(len(train)):
                curr_sim = (cosine_similarity(q, train.iloc[idx]['encoded_Q1']) + cosine_similarity(q, train.iloc[idx]['encoded_Q2']))/2
                if curr_sim > max_score:
                    max_index = idx
                    max_score = curr_sim


            tmp_list.append((max_index, max_score))


        idx_list.append(tmp_list)
    ref_list = []
    for i in idx_list:
        tmp_list = []
        for j in i:
            tmp_list.append(j[0])
        ref_list.append(tmp_list)
    test['reference_Q'] = [[] for _ in range(len(test))]
    test['reference_text'] =[[] for _ in range(len(test))]
    for idx, i in enumerate(ref_list):
        for j in i:
            test.iloc[idx]['reference_Q'].append(train.iloc[j]['질문_1'])

    for idx, i in enumerate(ref_list):
        for j in i:
            test.iloc[idx]['reference_text'].append(train.iloc[j]['concatenated_답변'])
    return train, test


In [ ]:
train = pd.read_csv('./data/train.csv', encoding='utf-8')
train = preprocess_train(train)

In [ ]:
test = pd.read_csv('./data/test.csv', encoding='utf-8')
train, test = test_split_question_and_RAG(train, test)

100%|██████████| 130/130 [00:20<00:00,  6.42it/s]


## test 데이터 prompt 만들기 & 질문 나누기

In [ ]:
def make_prompt(test):
    test['chat_sample'] = [[] for _ in range(len(test))]
    # Making Mistral 용 Prompt data

    for idx, row in test.iterrows():
        for i, j in zip(row['reference_text'], row['질문_split']):
            row['chat_sample'].append(f"너는 실내 인테리어 회사 상담직원이야 다음 질문에 대답해줘. 반드시 지켜야할 몇가지 규칙이 있어.\n1. 최대한 길게 대답해. \n2. 답변을 할 때 아래 정보 중 두 문장 이상을 활용해.\n3.정보와 질문이 관련없으면 정보를 사용하지말고 대답해.\n4. 길게 대답하는 것이 무엇보다 중요해.\n정보:{i}[ INST] {j} [/INST] ")

    # 질문 나누기
    test = test.explode('chat_sample').reset_index(drop=True)
    test['chat_num'] = test['id']
    return test

In [ ]:
test = make_prompt(test)

## Inference

In [ ]:
from datasets import Dataset, DatasetDict

test_dataset = Dataset.from_pandas(test)

In [ ]:
def extract_diarizationllm_outputs(batch):

    inputs = tokenizer(batch['chat_sample'], return_tensors="pt", truncation=False, padding=True).to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs,  max_new_tokens=256)
        decoded_outputs = [tokenizer.decode(o, skip_special_tokens=True) for o in outputs]

        decoded_llm = [o.split('[/INST]')[1].strip() for o in decoded_outputs]
        print(decoded_llm)
    return {'decoded_llm': decoded_llm}


In [ ]:
torch.cuda.empty_cache()
tokenizer.model_max_length = 4096

In [ ]:
test_llm = test_dataset.map(extract_diarizationllm_outputs, batched=True, batch_size=4)

Map:   0%|          | 0/196 [00:00<?, ? examples/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['방청페인트의 종류로는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 그리고 규산염페인트 등이 있습니다.', '원목사이딩은 가격이 비싸고 관리가 어렵고 습기에 약해 뒤틀림, 부서짐, 수축/팽창이 발생할 수 있다는 점이 단점입니다.', '도배지에 녹은 자국은 얼룩 형태로 나타나는 것을 의미합니다. 이러한 자국은 도배지 표면에 물이 침투하여 표면을 젖혀 녹이 생기는 것을 의미합니다.', '큐블럭은 일반 벽돌에 비해 가격이 비싸고 균열이 생기기 쉬우며 습기로 인해 하자가 발생할 수 있다는 점이 단점입니다.']


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['압출법 단열판은 습기에 강하고 곰팡이 및 세균 증식을 막을 수 있는 것이 장점입니다.', '철골구조를 사용하는 고층 건물에서 단열 효과를 높이기 위해서는 건물 외벽에 단열재를 부착하고, 단열재 사이에 고밀도 폴리우레탄 폼을 충전하여 단열재와 건물 외벽을 단단하게 결합하는 것이 좋습니다.', '일반적으로 도배 후 완전 건조되기 전까지 꼬임이 발생할 수 있습니다. 제조사에서 권장하는 건조기간(일반적으로 일주일)동안 충분한 시간을 확보하고, 건조기간 동안 온도와 습도에 유의하며, 일주일이 지나도 꼬임이 사라지지 않는다면 시공사에 연락해서 상세히 진단을 받아야 합니다.', '철근철골콘크리트 구조는 철근과 철골을 함께 사용하여 건물을 지은 구조체입니다. 이러한 구조는 철근의 인장 강도와 철골의 압축 강도를 결합하여 안정성을 높일 수 있습니다. 또한, 고층 건물에서는 아래층의 보나 기둥 단면이 커져야 하는 문제가 있습니다.']


...

['철근콘크리트 구조는 콘크리트의 압축 강도를 활용하면서도 인장 응력에 대한 취약점을 보완하기 위해 철근을 사용하는 구조입니다. 이러한 구조는 내구성, 내화 및 내진 성능이 우수하며, 철근의 사용으로 콘크리트의 인장응력 약점을 보완할 수 있습니다.', '일반적으로 도배지는 도배평수에 5를 나눈 값만큼 필요합니다. 예를 들어, 32평 아파트의 경우 도배평수는 80평(32*2.5)이며, 5로 나눈 값인 16롤만큼 필요합니다. 하지만 작업자의 숙련도, 작업환경, 집 구조 등에 따라 실제 필요한 양은 달라질 수 있습니다.', '벽에 구멍을 막는 가장 효과적인 방법은 합성 먼지나 석고를 사용하여 구멍을 메우고, 필요에 따라 패팅을 적용하는 것입니다.', '경질우레탄폼은 폴리소시아노레이트(Polyisocyanurate, PIR)를 단열소재로 사용하여 기존 폴리우레탄(Polyurethane, PUR)의 장점을 유지하면서 자기 소화성, 내열성, 저연성 등이 개선된 특징을 가지고 있습니다. 이로써 작은 두께로도 건축물의 에너지 절약 설계를 충족하며, 열전도를 완벽하게 차단하는 탁월한 기능을 가지고 있습니다.']


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['질석벽지는 아트월이나 현관 입구에 많이 사용되는 이유는 질감이 풍부하고 광택이 많아 고급스러운 분위기를 연출할 수 있기 때문입니다.', '시트 방수공사는 외단열 공법을 통해 에너지를 절약할 수 있는 장점이 있습니다.', '내진설계에서 안정성이 높은 순서로는 내진구조, 제진구조, 면진구조 순으로 안정성이 높습니다.', '내진구조란 지진이 발생했을 때 건물이나 다른 시설물이 붕괴되지 않도록 설계된 구조를 의미합니다. 내진벽과 같은 부재를 설치하여 건물을 보다 견고하게 만들어 지진에 대비하는 것이 내진구조의 목적입니다.']


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['분말 소화기를 사용할 때 주의할 점은 압력계의 눈금 위치가 초록색에 있는지 확인하고, 받침대를 사용하여 소화기 부식을 방지하고, 한 달에 한 번 흔들어 분말이 굳어짐을 방지하고, 직사광선이나 고온 다습한 장소를 피하고, 사용 후 내부 약제를 완전히 제거하는 것입니다.', '아파트 도배 평수는 일반적으로 분양 평수에 2.5를 곱하여 계산합니다.', '압출법 보온판은 습기에 강하고 곰팡이 및 세균 증식을 막을 수 있는 것이 가장 큰 장점입니다.', '평지붕의 누수 문제를 방지하기 위해 수성 벽체용 탄성 방수 도료를 사용하는 것은 효과적입니다. 이 도료는 벽체에 바르고 방수 기능을 제공하여 누수를 방지하고 벽체를 보호하는 역할을 합니다.']
['석고수정은 건물 구조적 결함, 지진, 외부 충격 등이 원인이 될 수 있습니다.', '새집증후군을 해결하려면 천연 소재와 포름알데히드 처리를 하지 않은 자재를 사용하고, 창문을 열어 자연스럽게 환기를 유지하며, 새로 이사하기 전에는 실내 난방 온도를 30도 이상으로 8시간 정도 유지하여 휘발성 유기물질을 제거해야 합니다.', '카페트의 기대수명은 일반적으로 6년입니다.', '오리지널징크는 다양한 패턴과 디자인을 적용할 수 있다는 것이 장점입니다.']


## 후처리 및 제출 파일(TEXT/VECTOR) 생성

In [ ]:
test_llm

Dataset({
    features: ['id', '질문', '질문_split', 'encoded', 'reference_Q', 'reference_text', 'chat_sample', 'chat_num', 'decoded_llm'],
    num_rows: 196
})

In [ ]:
def extract_ans(batch):
    sen_mo = SentenceTransformer('distiluse-base-multilingual-cased-v1')
    ans = sen_mo.encode(batch['decoded_llm'])
    return {'ans': ans}

def post_process(test_llm, submission):

    test_result = test_llm.to_pandas()
    test_result = test_result.groupby('id')['decoded_llm'].agg(lambda x: ' '.join(x)).reset_index()
    test_result_dataset = Dataset.from_pandas(test_result)
    test_result_dataset = test_result_dataset.map(extract_ans, batched=True, batch_size=64)

    #submission.iloc[:,1:] = test_result_dataset['ans'] # 1. Submission 추론이 원래 대회 규칙 대로
    submission.iloc[:,1:] = test_result_dataset['decoded_llm'] # 2. Submission 추론이 TEXT 형태로

    submission.to_csv('./data/final_submmit.csv', index=False)

    return submission


In [ ]:
post_process(test_llm, pd.read_csv('./data/sample_submission.csv', encoding='utf-8'))

Map:   0%|          | 0/130 [00:00<?, ? examples/s]